In [1]:
import cohere
import faiss
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [2]:
question = "Which Sydney beach should I visit?"



text = """

Sydney is world famous for beautiful beaches. These beaches offer different vibes and attractions, from bustling crowds and great surfing conditions to more tranquil and family-friendly environments. 



Bondi Beach: Bondi is perhaps the most famous beach in Sydney, if not Australia. It's known for its golden sands, vibrant atmosphere, and excellent surfing conditions. The Bondi to Coogee coastal walk offers stunning views of the coastline.



Manly Beach: Easily accessible by a scenic ferry ride from Circular Quay, Manly Beach is known for its laid-back atmosphere and family-friendly environment. The beach is great for swimming, surfing, and various water sports.



Cronulla Beach: Located in the Sutherland Shire, Cronulla offers a more relaxed atmosphere compared to some of the busier city beaches. It's a great spot for swimming, picnicking, and enjoying a range of seaside cafes and restaurants.



Bronte Beach: Situated between Bondi and Coogee, Bronte Beach is popular among both locals and visitors. It's a smaller, quieter beach with a beautiful park area and a natural rock pool that's ideal for swimming.



Tamarama Beach: Also known as "Glamarama" due to its popularity among the fashion-conscious crowd, Tamarama Beach is a smaller and more secluded option. It's surrounded by rocky cliffs and offers strong surf, making it a favorite among experienced surfers.

"""





In [7]:
# Split into a list of paragraphs
texts = text.split('\n\n')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts if t])
co = cohere.Client('api_key')

# Get the embeddings
response = co.embed(

    texts=texts.tolist(),

).embeddings




ApiError: status_code: 401, body: {'message': 'invalid api token'}

In [4]:


# Assuming 'response' is the list of embeddings
# Convert the response to a numpy array
embeds = np.array(response)

# Check the dimensions of the embeddings
print("Shape of embeddings:", embeds.shape)

# Ensure embeddings are float32 for Faiss
embeds = embeds.astype('float32')

# Get the dimension of the embeddings
d = embeds.shape[1]  # Dimension of the embeddings

# Create the Faiss index for L2 distance (Euclidean)
index = faiss.IndexFlatL2(d)

# Add vectors to the index
index.add(embeds)

# Save the index to a file
faiss.write_index(index, 'test.index')


Shape of embeddings: (6, 4096)


In [5]:


# Assuming 'response' is the list of embeddings
# Convert the response to a numpy array
embeds = np.array(response)

# Ensure embeddings are float32 for Faiss
embeds = embeds.astype('float32')

# Get the dimension of the embeddings
d = embeds.shape[1]  # Dimension of the embeddings

# Create the Faiss index for L2 distance (Euclidean)
index = faiss.IndexFlatL2(d)

# Add vectors to the index
index.add(embeds)

# Function to search for the nearest text
def search_text(query):
    # Get the query's embedding
    query_embed = co.embed(texts=[query]).embeddings
    query_embed = np.array(query_embed).astype('float32')  # Ensure float32 type

    # Retrieve the nearest neighbors using Faiss
    k = 10  # Number of nearest neighbors to retrieve
    distances, indices = index.search(query_embed, k)

    # Assuming 'texts' is a list of text items corresponding to the embeddings
    search_results = [texts[i] for i in indices[0]]

    return search_results

In [6]:
question = "What is my name?"
results = search_text(question)
print(results[0])

Tamarama Beach: Also known as "Glamarama" due to its popularity among the fashion-conscious crowd, Tamarama Beach is a smaller and more secluded option. It's surrounded by rocky cliffs and offers strong surf, making it a favorite among experienced surfers.
